In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img 
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
import os
import pandas as pd
import numpy as np

In [1]:
python3.12 -m pip install opencv-python opencv-python-headless


SyntaxError: invalid syntax (1317348273.py, line 1)

In [79]:
TRAIN_DIR='images/train'
TEST_DIR='images/test'

In [3]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

In [4]:
train=pd.DataFrame()
train['image'],train['label']=createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [6]:
test=pd.DataFrame()
test['image'],test['label']=createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [8]:
from tqdm.notebook import tqdm

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):   
        img = load_img(image, color_mode="grayscale", target_size=(48, 48))
        img = np.array(img) 
        features.append(img)  
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)  
    return features


In [10]:
train_features=extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [11]:
test_features=extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [12]:
x_train=train_features/255.0
x_test=test_features/255.0

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le=LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [16]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [17]:
model=Sequential()
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7,activation='softmax'))

C:\Users\tanub\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [80]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 809s 4s/step - accuracy: 0.2355 - loss: 1.8348 - val_accuracy: 0.2583 - val_loss: 1.8160
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 693s 3s/step - accuracy: 0.2455 - loss: 1.8173 - val_accuracy: 0.2621 - val_loss: 1.7806
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 746s 3s/step - accuracy: 0.2664 - loss: 1.7664 - val_accuracy: 0.2850 - val_loss: 1.7420
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 662s 3s/step - accuracy: 0.3392 - loss: 1.6544 - val_accuracy: 0.4416 - val_loss: 1.4700
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 732s 3s/step - accuracy: 0.3993 - loss: 1.5355 - val_accuracy: 0.4707 - val_loss: 1.3991
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 709s 3s/step - accuracy: 0.4326 - loss: 1.4580 - val_accuracy: 0.4808 - val_loss: 1.3466
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 871s 4s/step - accuracy: 0.4540 - loss: 1.4152 - val_accuracy: 0.5010 - val_loss: 1.3102
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 776s 3s/step - accuracy: 0.4662 - loss: 1.3766 - 

KeyboardInterrupt: 

In [104]:
model_json=model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [101]:
label=['angry','disgust','fear','happy','neutral','sad','surprise']

In [102]:
def ef(image):
    img=load_img(image,color_mode="grayscale")
    feature=np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0
    

In [103]:
image='images/train/angry/22.jpg'
print("Original image is of angry")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("Model prediction is:",pred_label)


Original image is of angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Model prediction is: angry


In [28]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [98]:
import os
print(os.path.isfile('emotiondetector.h5'))  # Should return True if the file exists


True


In [66]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Training the model
model.fit(x=x_train, y=y_train,
          batch_size=256,  # Increased batch size
          epochs=100,  # You can reduce this if the model starts to overfit
          validation_data=(x_test, y_test),
          callbacks=[early_stopping, reduce_lr])


Epoch 1/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1105s 10s/step - accuracy: 0.2365 - loss: 1.8503 - val_accuracy: 0.2583 - val_loss: 1.8186 - learning_rate: 0.0010
Epoch 2/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 774s 7s/step - accuracy: 0.2439 - loss: 1.8092 - val_accuracy: 0.2922 - val_loss: 1.7236 - learning_rate: 0.0010
Epoch 3/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 737s 7s/step - accuracy: 0.2983 - loss: 1.7191 - val_accuracy: 0.4042 - val_loss: 1.5735 - learning_rate: 0.0010
Epoch 4/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 680s 6s/step - accuracy: 0.3889 - loss: 1.5529 - val_accuracy: 0.4643 - val_loss: 1.3839 - learning_rate: 0.0010
Epoch 5/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 629s 6s/step - accuracy: 0.4390 - loss: 1.4475 - val_accuracy: 0.4992 - val_loss: 1.2957 - learning_rate: 0.0010
Epoch 6/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 646s 6s/step - accuracy: 0.4746 - loss: 1.3678 - val_accuracy: 0.5188 - val_loss: 1.2595 - learning_rate: 0.0010
Epoch 7/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 641s 6s/step - accuracy: 0.4953 -

KeyboardInterrupt: 

In [86]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\tanub\OneDrive\Desktop\coding\Face detection


In [88]:
model_json = model.to_json()
with open("emotiondetector.json", "w") as json_file:
    json_file.write(model_json)


In [95]:
import os
from tensorflow.keras.models import model_from_json

# Define the path to your JSON file
json_path = 'C:/Users/tanub/OneDrive/Desktop/coding/Face detection/emotiondetector.json'

# Load the JSON file
with open(json_path, 'r') as json_file:
    model_json = json_file.read()

# Load the model architecture from the JSON file
model = model_from_json(model_json)

# Compile the model (you'll need the weights file as well)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load the weights (make sure the weights file is also correctly referenced)
weights_path = 'C:/Users/tanub/OneDrive/Desktop/coding/Face detection/emotiondetector.h5'
model.load_weights("C:/Users/tanub/OneDrive/Desktop/coding/Face detection/emotiondetector.h5")



In [97]:
from keras.models import load_model
model = load_model("C:/Users/tanub/OneDrive/Desktop/coding/Face detection/emotiondetector.h5")
print("Model loaded successfully")


Model loaded successfully


In [1]:
import cv2
print(cv2.__version__)


4.11.0


In [2]:
try:
    import cv2
    print("OpenCV is installed correctly!")
except ModuleNotFoundError:
    print("ModuleNotFoundError: No module named 'cv2'")


OpenCV is installed correctly!


In [3]:
import sys
print(sys.path)


['c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312', '', 'c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages', 'c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32', 'c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\Pythonwin']


In [4]:
import cv2
print(cv2.__file__)


c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\cv2\__init__.py


In [5]:
import sys
sys.path.append(r"C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\cv2\__init__.py")
import cv2
print(cv2.__version__)


4.11.0


In [1]:
import cv2
print(cv2.data.haarcascades)


c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\cv2\data\
